In [195]:
!pip install -q langchain openai pypdf sentence_transformers InstructorEmbedding faiss-cpu

I0000 00:00:1728246074.133043  401877 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


## Second Approach

In [ ]:
import os
os.environ["GEMINI_API_KEY"]= "ENTER YOUR KEY"

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key='ENTER YOUR KEY')

In [204]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("BestPractices.pdf")  # Load your PDF file
data = loader.load()
print(data)

[Document(metadata={'source': 'BestPractices.pdf', 'page': 0}, page_content=' \n \n \n \n \n \n \n \n  \nEdited by  \nNadia SCIALABBA  \nClimate, Energy and Tenure Division (NRC) of the  \nFood and Agriculture Organization of the United Nation  (FAO)  \nCompiled by  \nIlka GOMEZ and Lisa THIVANT  \nTECA Team  – Research and Extension Division (DDNR) of the \nFood and Agriculture Organization of the United Nation  (FAO)  \n2015  \nTechnologies and practices for smallholder farmers  Training manual for  \nORGANIC \nAGRICU LTURE  \n \n'), Document(metadata={'source': 'BestPractices.pdf', 'page': 1}, page_content='1 \n \n  \nACKNOWLEDGEMENTS  \nThe production of this manual is a joint activity between the Climate, Energy and Tenure \nDivision ( NRC ) and the  Technologies and practices for smallholder farmers ( TECA ) Team \nfrom the Research and  Extension Division (DDNR ) of FAO Headquarters  in Rome, Italy.  \nThe realization of this manual has been possible thanks to the hard review, c

In [206]:
data = data[:50]

In [208]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=1000)
docs = text_splitter.split_documents(data)

print("Total number of Chunks: ", len(docs))  # Check how many chunks we have
for chunk in docs:
    print(chunk.page_content)

Total number of Chunks:  622
Edited by  
Nadia SCIALABBA  
Climate, Energy and Tenure Division (NRC) of the  
Food and Agriculture Organization of the United Nation  (FAO)  
Compiled by  
Ilka GOMEZ and Lisa THIVANT  
TECA Team  – Research and Extension Division (DDNR) of the 
Food and Agriculture Organization of the United Nation  (FAO)  
2015  
Technologies and practices for smallholder farmers  Training manual for  
ORGANIC 
AGRICU LTURE
1 
 
  
ACKNOWLEDGEMENTS  
The production of this manual is a joint activity between the Climate, Energy and Tenure 
Division ( NRC ) and the  Technologies and practices for smallholder farmers ( TECA ) Team 
from the Research and  Extension Division (DDNR ) of FAO Headquarters  in Rome, Italy.  
The realization of this manual has been possible thanks to the hard review, compilation and 
edition work of Nadia Scialabba, Natural Resources officer ( NRC ) and Ilka Gomez and Lisa 
Thivant, members of the TECA  Team. 
Special thanks are due to the Inter

In [210]:
type(chunk)

langchain_core.documents.base.Document

In [212]:
from langchain_chroma import Chroma

vectorstoredb = Chroma.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstoredb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key="ENTER YOUR KEY")


In [216]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define a system prompt
system_prompt = (
   "You are a Agricultural Organic Farming Expert. Provide clear, concise answers based on the provided context. "
    "If the information is not found in the context, state that the answer is unavailable. "
    "Use a maximum of three sentences."
    "\n\n"
    "{context}"
)

# Set up the prompt for the QA chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create the RAG chain
chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, chain)


In [220]:
response = rag_chain.invoke({"input": "What are some organic practices for managing water scarcity in agriculture?"})
print("Question: What are some organic practices for managing water scarcity in agriculture?")
print(response['answer'])

Question: What are some organic practices for managing water scarcity in agriculture?
Organic practices for managing water scarcity in agriculture include active water retention, water harvesting, and storing water. Improving water retention and infiltration into the soil are crucial steps for organic farmers to optimize the use of on-farm and natural resources sustainably. Soil composition and organic matter content play a significant role in the soil's ability to absorb and store water.


In [222]:
response = rag_chain.invoke({"input": "How do I start composting on my farm? What materials should I use?"})
print("Question: How do I start composting on my farm? What materials should I use?")
print(response['answer'])

Question: How do I start composting on my farm? What materials should I use?
To start composting on your farm, you will need plant materials and animal manures. These can include crop residues, kitchen scraps, grass clippings, and livestock manure. Layer these materials properly in a compost pile, ensuring a good balance of green (nitrogen-rich) and brown (carbon-rich) materials for successful decomposition.


In [224]:
response = rag_chain.invoke({"input": "Can mulching help me save water? How thick should the mulch be?"})
print("Question: Can mulching help me save water? How thick should the mulch be?")
print(response['answer'])

Question: Can mulching help me save water? How thick should the mulch be?
Mulching can help save water by reducing evaporation from the soil, allowing plants to use water more efficiently. The recommended thickness for mulch is typically 2-4 inches to effectively retain moisture and suppress weed growth.
